# Movie Recommendation System
### AML Assignement:
Name: Trina De
Roll No.: MDS201824


1. First we load the data


In [7]:
import sys
import numpy as np
import pandas as pd
import requests
import io

In [8]:
data=pd.read_csv("http://files.grouplens.org/datasets/movielens/ml-100k/u.data",delimiter='\t',header=None)

2. Creating a dictionary with the preferences of the users

In [9]:
df=pd.DataFrame(data)

In [10]:
df.columns=["UserID","MovieID","rating","timestamp"]

In [11]:
df.sort_values(by="UserID")

,UserID,MovieID,rating,timestamp
66567,1,55,5,875072688
62820,1,203,4,878542231
10207,1,183,5,875072262
9971,1,150,5,876892196
22496,1,68,4,875072688
9811,1,201,3,878542960
9722,1,157,4,876892918
9692,1,184,4,875072956
9566,1,210,4,878542909
9382,1,163,4,875072442


In [28]:
df.drop(["timestamp"],axis=1)

,UserID,MovieID,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
5,298,474,4
6,115,265,2
7,253,465,5
8,305,451,3
9,6,86,3


In [14]:
prefs={}
for index,row in df.iterrows():
    uid=row['UserID']
    mid=row['MovieID']
    rating=row['rating']
    prefs.setdefault(uid,{})
    prefs[uid][mid]=rating

3. Using the Pearson Correlation distance to find similar users


In [23]:
import math
def sim_pearson(dicti,p1,p2):
 sim_item={}
 for item in dicti[p1]:
     if item in dicti[p2]: 
        sim_item[item]=1
        
 n=len(sim_item)
 if n==0: 
     return 0

 sum_1=sum([dicti[p1][it] for it in sim_item])
 sum_2=sum([dicti[p2][it] for it in sim_item])

 sum1Sq=sum([pow(dicti[p1][it],2) for it in sim_item])
 sum2Sq=sum([pow(dicti[p2][it],2) for it in sim_item])
 
 pSum=sum([dicti[p1][it]*dicti[p2][it] for it in sim_item])
 
 num=pSum-(sum_1*sum_2/n)
 den=math.sqrt((sum1Sq-pow(sum_1,2)/n)*(sum2Sq-pow(sum_2,2)/n))
 if den==0: 
    return 0
 r=num/den
 return r

In [24]:
sim_pearson(prefs,1,2)

0.16084123285437085

4. Creating a dictionary that maps MovieID to Movie Name

In [31]:
movie_data = pd.read_csv("http://files.grouplens.org/datasets/movielens/ml-100k/u.item",encoding='latin-1',delimiter='|',header=None)
movie_data = pd.DataFrame(movie_data)
movie_dict={}
for index,row in movie_data.iterrows():
    movie_id=row[0]
    movie_name=row[1]
    movie_dict.setdefault(movie_id,"")
    movie_dict[movie_id]=movie_name

In [32]:
movie_dict

{1: 'Toy Story (1995)',
 2: 'GoldenEye (1995)',
 3: 'Four Rooms (1995)',
 4: 'Get Shorty (1995)',
 5: 'Copycat (1995)',
 6: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 7: 'Twelve Monkeys (1995)',
 8: 'Babe (1995)',
 9: 'Dead Man Walking (1995)',
 10: 'Richard III (1995)',
 11: 'Seven (Se7en) (1995)',
 12: 'Usual Suspects, The (1995)',
 13: 'Mighty Aphrodite (1995)',
 14: 'Postino, Il (1994)',
 15: "Mr. Holland's Opus (1995)",
 16: 'French Twist (Gazon maudit) (1995)',
 17: 'From Dusk Till Dawn (1996)',
 18: 'White Balloon, The (1995)',
 19: "Antonia's Line (1995)",
 20: 'Angels and Insects (1995)',
 21: 'Muppet Treasure Island (1996)',
 22: 'Braveheart (1995)',
 23: 'Taxi Driver (1976)',
 24: 'Rumble in the Bronx (1995)',
 25: 'Birdcage, The (1996)',
 26: 'Brothers McMullen, The (1995)',
 27: 'Bad Boys (1995)',
 28: 'Apollo 13 (1995)',
 29: 'Batman Forever (1995)',
 30: 'Belle de jour (1967)',
 31: 'Crimson Tide (1995)',
 32: 'Crumb (1994)',
 33: 'Desperado (1995)',
 34: '

5. Creating recommendations for users based on other users who are close to them in terms of similarity

In [38]:
def getRecommendations(person,pdict = prefs,similarity = sim_pearson,mdict = movie_dict):
 totals={}
 simSums={}
 for other in pdict:
    if other==person: 
        continue
    sim=similarity(pdict,person,other)
    if sim<=0: 
            continue
            
 for item in pdict[other]:
     if item not in pdict[person] or pdict[person][item]==0:
         totals.setdefault(item,0)
         totals[item]+=pdict[other][item]*sim
         simSums.setdefault(item,0)
         simSums[item]+=sim

 
 rankings=[(total/simSums[item],mdict[item]) for item,total in totals.items( )]
 rankings.sort( )
 rankings.reverse( )
 return rankings

In [41]:
getRecommendations(1)

[(5.0, 'Face/Off (1997)'),
 (5.0, 'Close Shave, A (1995)'),
 (5.0, 'City of Lost Children, The (1995)'),
 (4.0, 'Waiting for Guffman (1996)'),
 (4.0, 'Trainspotting (1996)'),
 (4.0, 'Liar Liar (1997)'),
 (4.0, "Jackie Chan's First Strike (1996)"),
 (4.0, 'Hercules (1997)'),
 (4.0, 'Air Force One (1997)'),
 (3.0, 'Heat (1995)'),
 (3.0, 'Happy Gilmore (1996)'),
 (2.0, 'Spawn (1997)')]